# Identifying VHHs for resynthesis (`fig-picking`)

## Yield chart

In [1]:
import pandas as pd
import altair as alt
from natsort import natsort_keygen

In [2]:
yields = pd.read_csv("../data/fig-picking/CG037c3.csv").set_index("well", drop=True).iloc[:,0].rename('yield')

In [3]:
df = pd.read_csv("../data/fig-picking/CG037c_map.csv", index_col=0).join(yields, on="well")

In [4]:
def pretty_hex(hex_str, sep=' ', n=2, length=6):
    from textwrap import wrap
    return sep.join(wrap(hex_str[0:length], n))

In [5]:
df['CDR3ID_short'] = df['CDR3ID'].str.slice(0, 6)
df['color'] = '#' + df['CDR3ID'].str.slice(0, 6)
df['antigen'] = df['antigen'].apply(lambda x: 'multiple porins' if ',' in x else ('OprM' if x == 'MexAB' else ('nonspecific' if x == 'bad' else x)))
df['antigen_index'] = df.groupby("antigen").cumcount()

In [6]:
antigens = df['antigen'].unique()

In [7]:
def make_yield_chart(df, yaxis=True, title=""):
    
    _y = {}
    if not yaxis:
        _y['axis'] = None
    return alt.Chart(df).mark_point(opacity=1, fillOpacity=1).encode(
        x=alt.X('antigen_index:O', axis=None),
        y = alt.Y('antigen', scale=alt.Scale(domain=antigens), **_y),
        color=alt.Color('color', scale=None),
        fill=alt.Fill('color', scale=None)
    ).properties(width={'step':10}, title=title)

Cloned rVHHs

In [8]:
chart1 = make_yield_chart(df, title="cloned")

Expressed rVHHs

In [9]:
min_yield = 100
df2 = df.query(f"`yield` > {min_yield} | well == 'H9'").copy()
df2['antigen_index'] = df2.groupby("antigen").cumcount()

chart2 = make_yield_chart(df2, yaxis=False, title="expressed")

rVHHs validated by FCM

In [10]:
validated_wells = ["E1","E2","D2","F1","D1","B1","H1","G1","G11","F11","E11","C6","H6","A6","B6","H5","G5","E6","A9","C9","C10","D9","D10","E9","F9","B7","C7","D7"]

df['validated_by_FCM'] = df['well'].isin(validated_wells)
df3 = df.query(f"validated_by_FCM & (`yield` > {min_yield})").copy()
df3['antigen_index'] = df3.groupby("antigen").cumcount()

In [11]:
base = alt.Chart(df3).encode(
    x=alt.X('antigen_index',axis=None),
    y=alt.Y('antigen',scale=alt.Scale(domain=antigens), axis=None)
)
points = base.mark_point(opacity=1, fillOpacity=1).encode(
    color=alt.Color('color', scale=None),
    fill=alt.Fill('color', scale=None)
)
text = base.mark_text(align='left', dx=10, dy=0.5, angle=45).encode(
    text='CDR3ID_short'
)
chart3 = (points + text).properties(width=300, title="validated by FCM")

Assemble figure panel

In [12]:
(chart1 | chart2 | chart3).resolve_axis(y='shared').configure_view(strokeWidth=0)

alt.HConcatChart(...)

Make rVHH icons for other figures

In [13]:
df2['label'] = df2['CDR3ID_short'] + ' (' + df2['well'] + ')'
base = alt.Chart(df2).encode(
    x=alt.X('antigen_index',axis=None),
    y=alt.Y('antigen',scale=alt.Scale(domain=antigens), axis=None)
)
points = base.mark_point(opacity=1, fillOpacity=1, size=25).encode(
    color=alt.Color('color', scale=None),
    fill=alt.Fill('color', scale=None)
)
text = base.mark_text(align='left', dx=6, size=6).encode(
    text='label'
)


(points + text).properties(width=600, title="rVHHs").configure_view(strokeWidth=0)

alt.LayerChart(...)